In [2]:
import pandas as pd
import re
import pandas as pd
from collections import Counter

In [3]:
# read excell shet to df
path = "./Ocean Governance and ocean economy governance matrix_IGOs.xlsx"
data = pd.read_excel(path, sheet_name="Sheet1")

# backup
df = data.copy(deep=True)

In [4]:
df.head()

,Institutions,Year,Scale,Spatial Jurisdiction,Subject Matter Jurisdiction,Source of Jurisdiction,Defined Objectives,Strategies,Defined inter-institutional Relationship,Vertical,Horizontal,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,Intergovernmental Oceanographic Commission (IOC),1960.0,Global,IOC jurisdiction is global delineated by the b...,The IOC's subject matter jurisdiction encompas...,The IOC’s authority is derived from its statut...,The objectives of the Intergovernmental Oceano...,IOC implements its objectives through series o...,IOC collaborates with UN specialized agencies ...,Vertical coordination within the IOC involves ...,Horizontal coordination within the IOC encompa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Food and Agriculture Organization of the Unite...,1945.0,Global,The FAO’s jurisdiction spans a vast array of m...,"FAO’s remit includes nutrition, food and agric...",The FAO’s jurisdiction is established through ...,"As stated in Article 1 of the Constitution, FA...",The FAO executes its objectives through a ser...,"As stated in its constitution, the FAO maintai...",The FAO’s vertical coordination involves colla...,Horizontal coordination within the FAO involve...,https://www.jus.uio.no/english/services/librar...,FAO https://www.fao.org/strategic-framework/en,NaN,NaN,NaN,NaN,NaN,NaN
2,Convention on the Intergovernmental Maritime C...,1948.0,Global,The IMO’s authority spans a global geographica...,The IMO's jurisdiction encompasses a comprehen...,The IMO's jurisdiction is established by the C...,"According to Part I, Article 1 of the Internat...",IMO implements its objectives and mandates thr...,The IMO collaborates with a diverse array of o...,Vertical coordination within IMO involves coll...,Horizontal coordination within the IMO involve...,https://wwwcdn.imo.org/localresources/en/About...,https://wwwcdn.imo.org/localresources/en/Knowl...,https://www.imo.org/en/MediaCentre/HotTopics/P...,NaN,NaN,NaN,NaN,NaN
3,Division for Ocean Affairs and the Law of the ...,1992.0,Global,UN DALOS does not have authority over any spec...,The UN DOALOS's mandate includes providing inf...,DOALOS derives its mandate from the United Nat...,According to the Secretary-General’s bulletin ...,. DOALOS) executes its objectives through a mu...,DOALOS collaborates with key organizations to ...,DOALOS engages in vertical coordination with v...,Horizontal coordination within DOALOS involves...,https://www.un.org/oceancapacity/projects,https://www.un.org/oceancapacity/tf,https://www.un.org/Depts/los/doalos_publicatio...,https://treaties.un.org/doc/source/docs/A_RES_...,https://documents-dds-ny.un.org/doc/UNDOC/GEN/...,https://documents-dds-ny.un.org/doc/UNDOC/GEN/...,https://www.un.org/depts/los/clcs_new/document...,https://unsceb.org/sites/default/files/2023-11...
4,Climate Change Secretariat,1992.0,Global,"The UNFCCC Secretariat operates globally, supp...",The UNFCCC Secretariat is responsible for faci...,The UNFCCC Secretariat derives its mandate fro...,The secretariat's duties under the Convention ...,The UNFCCC Secretariat executes its objectives...,The UNFCCC Secretariat maintains collaborative...,Vertical coordination within UNFCCC involves c...,Horizontal coordination within the UNFCCC invo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df[["Institutions", "Horizontal"]]

In [7]:
# Step 1: Cleaning the Data
def clean_text(text):
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = text.replace("United Nations", "UN").replace("scientific communities", "science sector").replace("Member States", "Member States")
    text = re.sub(r'collaboration across various sectors and stakeholders', 'collaboration with sectors and stakeholders', text)
    return text.strip()

df["Cleaned Practical Horizontal Coordination"] = df["Horizontal"].apply(clean_text)

# Step 2: Summarizing the Data
def summarize_horizontal(text):
    summary = ""
    if "collaboration" in text.lower():
        summary += "Collaborates with sectors like "
        if "governments" in text.lower(): summary += "governments, "
        if "science" in text.lower(): summary += "science, "
        if "private sector" in text.lower(): summary += "private sector, "
        if "civil society" in text.lower(): summary += "civil society, "
        summary = summary.rstrip(", ")
    if any(x in text.lower() for x in ["joint", "programmes", "projects"]): summary += " via joint initiatives"
    if any(x in text.lower() for x in ["guidelines", "standards"]): summary += " using guidelines"
    if "policy" in text.lower(): summary += " for policy coherence"
    return summary.strip()

df["Summarized Practical Horizontal Coordination"] = df["Cleaned Practical Horizontal Coordination"].apply(summarize_horizontal)

# Step 3: Assigning Themes (Using Key Phrases)
def assign_horizontal_themes(summary):
    themes = []
    # Map key phrases to themes
    if any(x in summary.lower() for x in ["governments", "science", "private sector", "civil society"]):  # From "Cross-sectoral collaboration for policy coherence"
        themes.append("Cross-Sectoral Collaboration")
    if "un agencies" in summary.lower():  # From "Inter-institutional partnerships for policy effectiveness"
        themes.append("Inter-Institutional Partnerships")
    if "guidelines" in summary.lower():  # From "Coordination mechanisms for integrated policy development"
        themes.append("Policy Integration Mechanisms")
    if any(x in text.lower() for x in ["report", "advocacy", "day"]):  # Proxy for "Collective responsibility for policy output"
        themes.append("Advocacy and Knowledge Sharing")
    if "joint" in summary.lower():  # From "Cooperative arrangements within and outside the institution"
        themes.append("Joint Program Implementation")
    return ", ".join(themes)

# Ensure key phrases influence theme assignment by checking original text for advocacy-related terms
for index, row in df.iterrows():
    text = row["Cleaned Practical Horizontal Coordination"]
    summary = row["Summarized Practical Horizontal Coordination"]
    themes = assign_horizontal_themes(summary)
    if any(x in text.lower() for x in ["report", "advocacy", "day"]) and "Advocacy and Knowledge Sharing" not in themes:
        themes += ", Advocacy and Knowledge Sharing" if themes else "Advocacy and Knowledge Sharing"
    df.at[index, "Horizontal Themes"] = themes

# Step 4: Validate Themes
def validate_themes(df):
    all_themes = df["Horizontal Themes"].str.split(", ").explode()
    theme_counts = Counter(all_themes)
    print("Horizontal Theme Frequency Counts:", theme_counts)

validate_themes(df)

# Step 5: Output Table
print("\nFinal Horizontal Coordination Table:")
print(df[["Institutions", "Cleaned Practical Horizontal Coordination", "Summarized Practical Horizontal Coordination", "Horizontal Themes"]].to_string(index=False))

# Save to CSV
df.to_csv("horizontal_coordination_analysis.csv", index=False)
print("\nData saved to 'horizontal_coordination_analysis.csv'")

Horizontal Theme Frequency Counts: Counter({'Cross-Sectoral Collaboration': 41, 'Advocacy and Knowledge Sharing': 41, 'Joint Program Implementation': 41, 'Policy Integration Mechanisms': 39, '': 3})

Final Horizontal Coordination Table:
                                                                                                      Institutions                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         